# SyriaTel Customer Churn ML Project

Project by:
- **Name**:  Julius Kinyua Njeri
- **Email**: juliusczar36@gmail.com
- **Date**: June 2024
- **Github Link**: https://github.com/CzarProCoder/SyriaTel_Customer_Churn_ML
- **LinkedIN**: https://www.linkedin.com/in/julius-kinyua
- **Twitter(X)**: https://x.com/Juliuskczar

# I: Business Understanding

# II: Data Understanding

# III: Data Preparation

# IV: Modeling

# V: Evaluation

# VI: Conclusion and Recommendations